# import the necessary libraries

In [5]:
import numpy as np
import plotly.graph_objects as go

# Creating Sample Data

In [6]:
# Generate random data
np.random.seed(42)  # For reproducibility
num_samples = 100

# Class 0 data (mean=[1, 5], std=[2, 1])
class0_mean = [1, 5]
class0_std = [2, 1]
class0_data = np.random.normal(loc=class0_mean, scale=class0_std, size=(num_samples // 2, 2))
class0_labels = np.zeros(num_samples // 2)

# Class 1 data (mean=[5, 2], std=[1, 2])
class1_mean = [5, 2]
class1_std = [1, 2]
class1_data = np.random.normal(loc=class1_mean, scale=class1_std, size=(num_samples // 2, 2))
class1_labels = np.ones(num_samples // 2)

# Combine data and labels
X = np.concatenate((class0_data, class1_data))
y = np.concatenate((class0_labels, class1_labels))

# Implementing the Naive Bayes Classifier

In [8]:
class GaussianNaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.means = {}
        self.stds = {}
        self.priors = {}

        for c in self.classes:
            X_c = X[y == c]
            self.means[c] = np.mean(X_c, axis=0)
            self.stds[c] = np.std(X_c, axis=0)
            self.priors[c] = X_c.shape[0] / X.shape[0]

    def predict(self, X):
        predictions = []
        for x in X:
            posteriors = {}
            for c in self.classes:
                prior = self.priors[c]
                likelihood = np.prod(self._gaussian_pdf(x, self.means[c], self.stds[c]))
                posteriors[c] = prior * likelihood
            predictions.append(max(posteriors, key=posteriors.get))
        return np.array(predictions)

    def _gaussian_pdf(self, x, mean, std):
        exponent = np.exp(-((x - mean) ** 2) / (2 * std ** 2))
        return (1 / (np.sqrt(2 * np.pi) * std)) * exponent

# Training and Evaluating the Classifier

In [9]:
# Create and train the classifier
gnb = GaussianNaiveBayes()
gnb.fit(X, y)

# Make predictions
predictions = gnb.predict(X)

# Evaluate accuracy
accuracy = np.mean(predictions == y)
print(f"Accuracy: {accuracy}")

Accuracy: 0.97


# Visualizing the Decision Boundary (3D)

In [10]:
# Create a meshgrid
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))
Z = gnb.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Create a 3D surface plot
fig = go.Figure(data=[go.Surface(x=xx, y=yy, z=Z)])
fig.update_layout(scene=dict(xaxis_title='Feature 1', yaxis_title='Feature 2', zaxis_title='Class'))
fig.show()